# Python Execution Time Tracking

A simple python class for tracking how much time is spent in functions.
It allows you to track:
- the *total time* spent in a function i.e., the time delta between entering and exiting a function;
- the *partial time* spent in a function i.e., the total time minus the total time spent in **time-tracked functions** called inside the function.


## Dependencies

Only standard python libraies:
- `time`: used to get the current unix time
- `functools`: to properly wrap functions when decorating them

In [1]:
import time
from functools import wraps

## Logger module


It is a required argument of the TimeTracker module. It must provides a `log` function that log a given json message. The code below implement a basic Logger module.


In [2]:
class Logger:
    def log(self, message):
        print('[TimeTracking] {function_name} {args} {kwargs} - total: {total_time:3.6f} s - partial: {partial_time:3.6f} s'.format(**message))

## The TimeTracker module

Arguments when initialization:
- [Required] `logger`: an instance that have a `log` function attribue;
- [Optional] `do_time_tracking`: a boolean that enables the time tracker if True. Default value is True;
- [Optional] `isolated`: the time tracker uses global variables if `isolated` is False. It allows time_tracker initialized in different files to share the same variables hence tracking consistent times. See below for an example.

In [3]:
_time_tracker_global_entering_times = []
_time_tracker_global_total_times = [0]

class TimeTracker:
    def __init__(self, logger, do_time_tracking=True, isolated=True):
        self.logger = logger
        self.do_time_tracking = do_time_tracking

        if isolated:
            self.entering_times = []
            self.total_times = [0]
        else:
            self.entering_times = _time_tracker_global_entering_times
            self.total_times = _time_tracker_global_total_times
            
    def _enter(self):
        self.entering_times.append(time.time())
        self.total_times.append(0)
 
    def _exit(self, function_name, args, kwargs):
        enter_time = self.entering_times.pop()
        inner_total_time = self.total_times.pop()

        exit_time = time.time()
        total_time = exit_time - enter_time
        partial_time = total_time - inner_total_time
        
        self.total_times[-1] += total_time
        
        self.logger.log({
            'function_name': function_name,
            'args': args,
            'kwargs': kwargs,
            'total_time': total_time,
            'partial_time': partial_time,
        })
        
    def __str__(self):
        return 'entering_times: {entering_times}, total_times: {total_times}'.format(
            entering_times=self.entering_times,
            total_times=self.total_times,
        )
    
    def time_track(self):
        def _time_track(function):
            @wraps(function)
            def wrapped(*args, **kwargs):
                self._enter()
                try:
                    result = function(*args, **kwargs)
                finally:
                    self._exit(function.__name__, args, kwargs)

                return result
            return wrapped

        return _time_track if self.do_time_tracking else lambda function: function

### Initialization

In [4]:
logger = Logger()
time_tracker = TimeTracker(logger, True)

print(time_tracker)

entering_times: [], total_times: [0]


## Examples

### 1. Basic example

A function `boo` call a function `dodo` twice.
Both function are decorated with the `time_tracker.time_track` function. Note that the two `dodo` total execution times are taken into account in `boo` partial execution time.

In [5]:
@time_tracker.time_track()
def boo(x, y):
    time.sleep(0.2)
    return dodo(x, 0.3) + dodo(y, 0.4)

@time_tracker.time_track()
def dodo(x, i):
    time.sleep(i)
    return x

boo(1, 2)

[TimeTracking] dodo (1, 0.3) {} - total: 0.300370 s - partial: 0.300370 s
[TimeTracking] dodo (2, 0.4) {} - total: 0.400543 s - partial: 0.400543 s
[TimeTracking] boo (1, 2) {} - total: 0.901676 s - partial: 0.200763 s


3

### 2. Recursive functions

In [6]:
@time_tracker.time_track()
def f(x, i):
    time.sleep(0.1)
    if i == 0:
        return x
    else:
        return 3 * g(x, i) + 1

@time_tracker.time_track()
def g(x, i):
    time.sleep(0.3)
    res = f(x-1, i-1) / 2
    time.sleep(0.4)
    return res

f(1, 2)

[TimeTracking] f (-1, 0) {} - total: 0.100327 s - partial: 0.100327 s
[TimeTracking] g (0, 1) {} - total: 0.801705 s - partial: 0.701378 s
[TimeTracking] f (0, 1) {} - total: 0.902204 s - partial: 0.100498 s
[TimeTracking] g (1, 2) {} - total: 1.603253 s - partial: 0.701049 s
[TimeTracking] f (1, 2) {} - total: 1.703762 s - partial: 0.100509 s


0.25

### 3. Another recursive function: the fibonacci function

In [7]:
@time_tracker.time_track()
def fibo(n):
    if n == 0 or n == 1:
        return 1
    else:
        return fibo(n-1) + fibo(n-2)
    
fibo(5)

[TimeTracking] fibo (1,) {} - total: 0.000008 s - partial: 0.000008 s
[TimeTracking] fibo (0,) {} - total: 0.000020 s - partial: 0.000020 s
[TimeTracking] fibo (2,) {} - total: 0.002459 s - partial: 0.002431 s
[TimeTracking] fibo (1,) {} - total: 0.000011 s - partial: 0.000011 s
[TimeTracking] fibo (3,) {} - total: 0.004395 s - partial: 0.001925 s
[TimeTracking] fibo (1,) {} - total: 0.000005 s - partial: 0.000005 s
[TimeTracking] fibo (0,) {} - total: 0.000005 s - partial: 0.000005 s
[TimeTracking] fibo (2,) {} - total: 0.000167 s - partial: 0.000157 s
[TimeTracking] fibo (4,) {} - total: 0.004750 s - partial: 0.000187 s
[TimeTracking] fibo (1,) {} - total: 0.000004 s - partial: 0.000004 s
[TimeTracking] fibo (0,) {} - total: 0.000005 s - partial: 0.000005 s
[TimeTracking] fibo (2,) {} - total: 0.000224 s - partial: 0.000215 s
[TimeTracking] fibo (1,) {} - total: 0.000005 s - partial: 0.000005 s
[TimeTracking] fibo (3,) {} - total: 0.000411 s - partial: 0.000182 s
[TimeTracking] fibo 

8

### 4. Time track functions raising exceptions

In [8]:
@time_tracker.time_track()
def error_fun():
    time.sleep(0.2)
    raise Exception('Booo')

@time_tracker.time_track()
def error_fun_caller():
    time.sleep(0.1)
    error_fun()
    
error_fun_caller()

[TimeTracking] error_fun () {} - total: 0.200417 s - partial: 0.200417 s
[TimeTracking] error_fun_caller () {} - total: 0.301006 s - partial: 0.100589 s


Exception: Booo

### Time trackers using global variables

But default, different time trackers will not be able to take time-tracked sub-calls into account. For instance:

In [9]:
## time_tracker_1 defined in some file
time_tracker_1 = TimeTracker(logger)

@time_tracker_1.time_track()
def boo(x, y):
    time.sleep(0.2)
    return dodo(x, 0.3) + dodo(y, 0.4)

## time_tracker_2 defined in some other file
time_tracker_2 = TimeTracker(logger)

@time_tracker_2.time_track()
def dodo(x, i):
    time.sleep(i)
    return x

boo(1, 2)

[TimeTracking] dodo (1, 0.3) {} - total: 0.300512 s - partial: 0.300512 s
[TimeTracking] dodo (2, 0.4) {} - total: 0.400593 s - partial: 0.400593 s
[TimeTracking] boo (1, 2) {} - total: 0.901966 s - partial: 0.901966 s


3

To aleviate this, you should set the `isolated` optional argument to True. For instance:

In [10]:
## time_tracker_1 defined in some file
time_tracker_1 = TimeTracker(logger, isolated=False)

@time_tracker_1.time_track()
def boo(x, y):
    time.sleep(0.2)
    return dodo(x, 0.3) + dodo(y, 0.4)

## time_tracker_2 defined in some other file
time_tracker_2 = TimeTracker(logger, isolated=False)

@time_tracker_2.time_track()
def dodo(x, i):
    time.sleep(i)
    return x

boo(1, 2)

[TimeTracking] dodo (1, 0.3) {} - total: 0.300505 s - partial: 0.300505 s
[TimeTracking] dodo (2, 0.4) {} - total: 0.400618 s - partial: 0.400618 s
[TimeTracking] boo (1, 2) {} - total: 0.901944 s - partial: 0.200821 s


3